In [ ]:
# coding: utf-8
import pickle
import pandas as pd
import numpy as np

from keras import models
from keras import layers



In [ ]:
with open('../../data/transformed_data.pkl','rb') as f:
    transformed_data = pickle.load(f)
    f.close()

with open('../../data/targets.pkl','rb') as f:
    targets = pickle.load(f)
    f.close()

Add bias node to the data

In [ ]:
transformed_data = np.c_[np.ones((transformed_data.shape[0], 1)), transformed_data]

Create a stratified test and train set, ensuring that there is a good proportion of targets and non-targets in the test/training set.

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits = 1, test_size=0.2, random_state=1337)
for train_index, test_index in split.split(transformed_data, targets):
    train_index = train_index
    test_index=test_index


In [ ]:
print('test_m =', len(test_index))
print('test_m =', len(train_index))
print('proportion of targets =',sum(targets[test_index])/len(targets[test_index]))
print('proportion of targets =',sum(targets[train_index])/len(targets[train_index]))

train_X = transformed_data[train_index]
train_y = targets[train_index]
test_X = transformed_data[test_index]
test_y = targets[test_index]

# Try to solve label shape error

from sklearn.utils import column_or_1d 
train_y = column_or_1d(train_y)
test_y = column_or_1d(test_y)

In [ ]:
m = train_X.shape[0]
n = train_X.shape[1]

In [ ]:
network = models.Sequential()
network.add(layers.Dense(200, activation='elu', input_shape=(n,)))
network.add(layers.Dense(200, activation='elu', input_shape=(n,)))
network.add(layers.Dense(2, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

from keras.utils import to_categorical

train_y = to_categorical(train_y)
test_y = to_categorical(test_y)

network.fit(train_X, train_y, epochs=5, batch_size=128)

In [ ]:
test_loss, test_acc = network.evaluate(test_X, test_y)

print('test_acc:', test_acc)

In [ ]:
test_pred = network.predict_classes(test_X)


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(test_y, test_pred))